In [49]:
import pandas as pd
device_df = pd.read_csv('device.csv')
logon_df = pd.read_csv('logon.csv')
psycho_df = pd.read_csv('psychometric.csv', index_col=0)

In [39]:
import math
import random
device_df.assign(insider=0) # insiderのラベル追加
insiders_df = pd.read_csv('insiders.csv')
insiders = insiders_df[insiders_df['dataset'] == 4.2]['user'].tolist()
device_df["insider"] = device_df["user"].apply(lambda x : 0 if x  in insiders else  1)
nin_insider_list = list(device_df[device_df['insider'] == 1].index) 
del_rows = random.sample(nin_insider_list, math.floor(0.87*len(nin_insider_list))) # insiderとnot insiderの割合をいい感じにするためにnot insiderを削除
device_df = device_df.drop(del_rows)
device_df

,id,date,user,pc,activity,insider
4,{L7P2-G4PX02RX-7999GYOY},01/02/2010 08:04:26,IIW0249,PC-0843,Disconnect,1
8,{H9X8-R0QB50BG-6009OAKA},01/02/2010 08:25:19,MOH0273,PC-6699,Connect,1
15,{G8U2-N0YV97HS-0321CNBR},01/02/2010 09:22:55,BQS0525,PC-0269,Connect,1
23,{J3R3-W0NN51HK-0256NDFA},01/02/2010 09:45:49,JCR0172,PC-6713,Connect,1
37,{F9L3-Y8ZE73LG-8974FHET},01/02/2010 10:30:55,MOH0273,PC-6699,Connect,1
...,...,...,...,...,...,...
405339,{W5Y1-T6TL17JR-7106BAFE},05/16/2011 18:28:28,BAL0044,PC-9800,Connect,1
405345,{U6Y2-J9DD01DS-0481JYUP},05/16/2011 18:45:34,BRM0995,PC-0768,Connect,1
405348,{E5J3-I9GU99EL-0381WLDK},05/16/2011 18:52:27,AJF0370,PC-7375,Connect,1
405351,{Z8Z9-Y6HM60ML-6100DENH},05/16/2011 19:00:10,HRL0540,PC-1117,Disconnect,1


In [40]:
import math
import random
logon_df.assign(insider=0) # insiderのラベル追加
insiders_df = pd.read_csv('insiders.csv')
insiders = insiders_df[insiders_df['dataset'] == 4.2]['user'].tolist()
logon_df["insider"] = logon_df["user"].apply(lambda x : 0 if x  in insiders else  1)
nin_insider_list = list(logon_df[logon_df['insider'] == 1].index) 
del_rows = random.sample(nin_insider_list, math.floor(0.87*len(nin_insider_list))) # insiderとnot insiderの割合をいい感じにするためにnot insiderを削除
logon_df = logon_df.drop(del_rows)
logon_df

,id,date,user,pc,activity,insider
2,{U6Q3-U0WE70UA-3770UREL},01/02/2010 06:53:04,LRR0148,PC-4124,Logon,1
10,{S6C5-Q8LQ18VJ-5964MXZD},01/02/2010 07:24:53,LRR0148,PC-4124,Logoff,1
14,{P3Y3-J4EZ51PP-3037PQQN},01/02/2010 07:37:00,ABC0174,PC-5156,Logon,0
34,{Q0C9-O0SK97VF-2901JTEV},01/02/2010 08:07:00,YIC0195,PC-9353,Logon,1
41,{I1S2-K5OX13ST-9590PKMV},01/02/2010 08:20:00,FKK0055,PC-0435,Logon,1
...,...,...,...,...,...,...
854829,{X2T3-B3FS97QK-0418OCCH},05/16/2011 21:39:17,ACC0042,PC-2231,Logoff,1
854848,{S6D0-G0NQ69OF-2637QOML},05/16/2011 23:37:23,QNP0216,PC-0164,Logon,1
854849,{W7T3-X0YN28PN-9538ZMMN},05/16/2011 23:44:00,KJP0371,PC-9747,Logoff,1
854856,{P2S0-P1JS62IF-2767BEPH},05/17/2011 05:25:33,IRM0931,PC-7188,Logon,1


In [50]:
device_df["activity"] = device_df["activity"].apply(lambda x : 0 if x == "Connect" else  1) # activityを0, 1に変換
device_df["date"] = pd.to_datetime(device_df["date"]) # 日時をdatetime型に変換

In [51]:
logon_df["activity"] = logon_df["activity"].apply(lambda x : 0 if x == "Logon" else  1) # activityを0, 1に変換
logon_df["date"] = pd.to_datetime(logon_df["date"]) # 日時をdatetime型に変換

In [52]:
def get_timezone(hour: int) -> int:
        if 0 <= hour and hour < 6:
            return 0
        elif 6 <= hour and hour < 12:
            return 1
        elif 12 <= hour and hour < 18:
            return 2
        else:
            return 3

users = device_df['user'].unique()
d = {}
for user in users:
    d[user] = {} # connect, disconnectを記録
    for index, row in device_df[device_df['user']== user].iterrows():
        ym = str(row['date'].year)+str(row['date'].month)
        hour = row['date'].hour
        if(row['activity'] == 0):
            if ym not in d[user]:
                d[user][ym] = [0]*16
            tz = get_timezone(hour)
            d[user][ym][tz] += 1 
        if(row['activity'] == 1):
            if ym not in d[user]:
                d[user][ym] = [0]*16
            tz = get_timezone(hour)
            d[user][ym][tz+4] += 1 

users = logon_df['user'].unique()
for user in users:
    if user not in d:
        d[user] = {} # logon, logoffを記録
    for index, row in logon_df[logon_df['user']== user].iterrows():
        ym = str(row['date'].year)+str(row['date'].month)
        hour = row['date'].hour
        if(row['activity'] == 0):
            if ym not in d[user]:
                d[user][ym] = [0]*16
            tz = get_timezone(hour)
            d[user][ym][tz+8] += 1 
        if(row['activity'] == 1):
            if ym not in d[user]:
                d[user][ym] = [0]*16
            tz = get_timezone(hour)
            d[user][ym][tz+12] += 1 

In [56]:
# USBの挿抜に関する列を追加

main_df = pd.DataFrame({'user':[], 'O': [], 'C': [], 'E': [], 'A': [], 'N': [], 
                        'dev_con_0to6': [], 'dev_con_6to12': [], 'dev_con_12to18': [], 'dev_con_18to24': [],
                        'dev_dis_0to6': [], 'dev_dis_6to12': [], 'dev_dis_12to18': [], 'dev_dis_18to24': [], 
                        'pc_on_0to6': [], 'pc_on_6to12': [], 'pc_on_12to18': [], 'pc_on_18to24': [],
                        'pc_off_0to6': [], 'pc_off_6to12': [], 'pc_off_12to18': [], 'pc_off_18to24': [],
                       }, index=[])

for user, v in d.items():
    for ym, vv in v.items():
        row = psycho_df[psycho_df['user_id']==user]
        main_df = main_df.append({'user': user, 'O': row['O'][0], 'C':  row['C'][0], 'E':  row['E'][0], 'A':  row['A'][0], 'N':  row['N'][0],  
                        'dev_con_0to6':  vv[0], 'dev_con_6to12': vv[1], 'dev_con_12to18': vv[2], 'dev_con_18to24': vv[3],
                        'dev_dis_0to6': vv[4], 'dev_dis_6to12': vv[5], 'dev_dis_12to18': vv[6], 'dev_dis_18to24': vv[7], 
                        'pc_on_0to6': vv[8], 'pc_on_6to12': vv[9], 'pc_on_12to18': vv[10], 'pc_on_18to24': vv[11],
                        'pc_off_0to6': vv[12], 'pc_off_6to12': vv[13], 'pc_off_12to18': vv[14], 'pc_off_18to24': vv[15],
                       }, ignore_index=True)
main_df

,user,O,C,E,A,N,dev_con_0to6,dev_con_6to12,dev_con_12to18,dev_con_18to24,...,dev_dis_12to18,dev_dis_18to24,pc_on_0to6,pc_on_6to12,pc_on_12to18,pc_on_18to24,pc_off_0to6,pc_off_6to12,pc_off_12to18,pc_off_18to24
0,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,83.0,115.0,0.0,...,122.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0
1,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,96.0,85.0,0.0,...,91.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,24.0,0.0
2,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,88.0,89.0,0.0,...,92.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,23.0,0.0
3,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,76.0,105.0,0.0,...,109.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,23.0,0.0
4,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,97.0,88.0,0.0,...,92.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15894,RHY0079,20.0,19.0,14.0,21.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,21.0,1.0,0.0,0.0,0.0,1.0,21.0
15895,RHY0079,20.0,19.0,14.0,21.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,21.0,1.0,0.0,0.0,1.0,1.0,20.0
15896,RHY0079,20.0,19.0,14.0,21.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,23.0
15897,RHY0079,20.0,19.0,14.0,21.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,20.0


In [57]:
main_df.assign(insider=0) # insiderのラベル追加
insiders_df = pd.read_csv('insiders.csv')
insiders = insiders_df[insiders_df['dataset'] == 4.2]['user'].tolist()
main_df["insider"] = main_df["user"].apply(lambda x : 0 if x  in insiders else  1)
nin_insider_list = list(main_df[main_df['insider'] == 1].index) 
del_rows = random.sample(nin_insider_list, math.floor(0.87*len(nin_insider_list))) # insiderとnot insiderの割合をいい感じにするためにnot insiderを削除
main_df = main_df.drop(del_rows)
main_df

,user,O,C,E,A,N,dev_con_0to6,dev_con_6to12,dev_con_12to18,dev_con_18to24,...,dev_dis_18to24,pc_on_0to6,pc_on_6to12,pc_on_12to18,pc_on_18to24,pc_off_0to6,pc_off_6to12,pc_off_12to18,pc_off_18to24,insider
2,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,88.0,89.0,0.0,...,0.0,0.0,23.0,0.0,0.0,0.0,0.0,23.0,0.0,1
8,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,94.0,85.0,0.0,...,0.0,0.0,22.0,0.0,0.0,0.0,0.0,22.0,0.0,1
14,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,93.0,115.0,0.0,...,0.0,0.0,26.0,0.0,0.0,0.0,0.0,26.0,0.0,1
16,MOH0273,39.0,40.0,47.0,50.0,26.0,0.0,43.0,53.0,0.0,...,0.0,0.0,12.0,0.0,0.0,0.0,0.0,12.0,0.0,1
41,IIW0249,39.0,36.0,43.0,38.0,28.0,0.0,76.0,150.0,0.0,...,0.0,1.0,26.0,2.0,1.0,1.0,1.0,28.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15876,TLH0648,32.0,35.0,18.0,24.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,20.0,1
15878,TLH0648,32.0,35.0,18.0,24.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,20.0,1
15879,TLH0648,32.0,35.0,18.0,24.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,23.0,1
15887,RHY0079,20.0,19.0,14.0,21.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,22.0,1


In [58]:
df_ins = (main_df["insider"] == 0)
print("insider :", df_ins.sum()) # 内部不正者のログ件数をカウント
df_niins = (main_df["insider"] == 1)
print("not insider :", df_niins.sum()) # 内部不正者でない人のログ件数をカウント
print(df_ins.sum()/(df_ins.sum()+df_niins.sum())) # 全体における内部不正者のログ件数の割合

insider : 760
not insider : 1969
0.2784902894833272


In [60]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

X = main_df[['O', 'C', 'E', 'A', 'N', 'dev_con_0to6', 'dev_con_6to12', 'dev_con_12to18', 'dev_con_18to24', 'dev_dis_0to6', 'dev_dis_6to12', 'dev_dis_12to18', 'dev_dis_18to24', 'pc_on_0to6', 'pc_on_6to12', 'pc_on_12to18', 'pc_on_18to24', 'pc_off_0to6', 'pc_off_6to12', 'pc_off_12to18', 'pc_off_18to24']]
Y = main_df['insider']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする

# ロジスティック回帰で学習
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, Y_train)
print("coefficient = ", lr.coef_)
print("intercept = ", lr.intercept_)

Y_pred = lr.predict(X_test)
np.set_printoptions(threshold=np.inf)
print(Y_pred)

coefficient =  [[-0.00811399 -0.0147414   0.01349019  0.03293019  0.00601434 -1.8426328
  -0.79240534 -0.10645485  0.17172625  1.80135312  0.79577801  0.12998358
  -0.17681121 -0.19468721 -0.05885516  0.07327601 -0.12799194  0.11971723
  -0.02095433  0.11640385  0.1111035 ]]
intercept =  [-0.59543418]
[1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1
 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1

In [61]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# モデルの評価
print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred))
print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))

confusion matrix = 
 [[ 46 116]
 [ 16 368]]
accuracy =  0.7582417582417582
precision =  0.7603305785123967
recall =  0.9583333333333334
f1 score =  0.8479262672811061
